# PREPROCESARE VENUE
### 1. Extrage venues in litere mici
### 2. Elimina punctuatia 
### 3. Elimina cuvintele care nu sunt in engleza
### 4. Elimina stopwords
### 5. Calculez numarul de aparitii pentru fiecare cuvant
### 6. Elimin cuvinte cu aparitie mai mare de 15 si de 10

In [1]:
import json # pentru fisier
import string # pentru lower si punctuation
from nltk.corpus import wordnet # pentru verificare limbii
from nltk.corpus import stopwords # pentru eliminare stopwords
import itertools # pentru sortare

In [17]:
def extrage_venues(file):
    venues = []
    readFile = open(file,'r',encoding='utf-8')

    for line in readFile:
        current_paper = json.loads(line)
        current_venue = current_paper['venue']
        venues.append(current_venue.lower())
    readFile.close()
    return venues

def elimina_punctuatia(venues):
    new_venues = []
    
    for venue in venues:
        
        word = ""
        new_current_venue = []
        
        for ch in venue:
            if ch == ' ' or ch =='-':
                new_current_venue.append(word)
                word = ""
            else:
                if ch not in string.punctuation:
                    word += ch
        new_current_venue.append(word)
        new_venues.append(new_current_venue)
        
    return new_venues

def elimina_non_eng(venues):
    new_venues = []
    
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if wordnet.synsets(elem):
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues

def elimina_stopwords(venues):
    new_venues = []
    stop_words = set(stopwords.words('english')) 
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if elem not in stop_words:
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues

def calculeaza_nr_aparitii(venues):
    # Calculez numarul total de cuvinte
    words_dict = {}
    word_list = []
    for venue in venues:
        for elem in venue:
            try:
                words_dict[elem] += 1
            except:
                words_dict[elem] = 1
                word_list.append(elem)
                
    return words_dict, word_list

def elimina_cuv_dupa_aparitie(word_dict, word_list,frecv):
    new_word_list = []
    
    for word in word_list:
        if word_dict[word] < frecv:
            new_word_list.append(word)
    return new_word_list

def elimina_venue(venues,word_list_reduced):
    new_venues = []
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if elem in word_list_reduced:
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues
    

## Functii auxiliare

In [3]:
# Functii auxiliare
def print_ll(listaDeListe, numeFisier):
    numeFisier += ".json"
    wFile = open(numeFisier,'w')
    wFile.write(json.dumps(listaDeListe))
    wFile.close()

def preety_print_ll(listaDeListe, numeFisier):
    numeFisier += "_pp.txt"
    wFile = open(numeFisier,'w')
    for lista in listaDeListe:
        for elem in lista:
            wFile.write(elem)
            wFile.write(' ')
        wFile.write('\n')
    wFile.close()

def preety_print_l(lista, numeFisier):
    numeFisier += "_ppl.txt"
    wFile = open(numeFisier,'w')
    for elem in lista:
        wFile.write(elem)
        wFile.write('\n')
    wFile.close()
    
def print_dict(dict_, numeFisier):
    numeFisier += ".json"
    wf = open(numeFisier,"w")
    wf.write(json.dumps(dict_))
    wf.close()
    
def eliminare_duplicate_si_vide(lista):
    lista.sort()
    sort_lista = list(lista for lista,_ in itertools.groupby(lista))
    if sort_lista[0] == []:
        return sort_lista[1:]
    return sort_lista
    
    
def maxim_aparitii(word_list, word_dict):
    max_ap = 0
    for elem in word_list:
        if word_dict[elem] > max_ap:
            max_ap = word_dict[elem]
    return max_ap

def sortare_dictionar(word_dict):
    word_dict_sorted = {}
    for w in sorted(word_dict, key=word_dict.get, reverse=True):
        word_dict_sorted[w] = word_dict[w]
    return word_dict_sorted

def append_listeDeListe(new_list, old_list):
    for elem in new_list:
        old_list.append(elem)
    return old_list

In [4]:
venues0 = extrage_venues('../../dblp-ref/dblp-final-0.json')
venues1 = extrage_venues('../../dblp-ref/dblp-final-1.json')
venues2 = extrage_venues('../../dblp-ref/dblp-final-2.json')
venues3 = extrage_venues('../../dblp-ref/dblp-final-3.json')

venues = append_listeDeListe(venues0,venues1)
venues = eliminare_duplicate_si_vide(venues)


venues = append_listeDeListe(venues2,venues)
venues = eliminare_duplicate_si_vide(venues)

venues = append_listeDeListe(venues3,venues)
venues = eliminare_duplicate_si_vide(venues)

print(len(venues))

4264


## Preprocesare venues pasi

In [5]:
venues = elimina_punctuatia(venues)
venues = elimina_non_eng(venues)
venues = elimina_stopwords(venues)
venues = eliminare_duplicate_si_vide(venues)
print(len(venues))

3810


In [6]:
print_ll(venues,"all_venues")

In [7]:
word_dict, word_list = calculeaza_nr_aparitii(venues)

In [8]:
print(len(word_list))
print(len(word_dict))

1542
1542


In [9]:
word_dict_sorted = sortare_dictionar(word_dict)
print(len(word_dict_sorted))

1542


In [10]:
word_list_reduced15 = elimina_cuv_dupa_aparitie(word_dict, word_list,15)
word_list_reduced10 = elimina_cuv_dupa_aparitie(word_dict, word_list,10)
print(len(word_list_reduced15))
print(len(word_list_reduced10))

1380
1306


## Elimina cuvintele

In [16]:
venues_reduced_15 = elimina_venue(venues,word_list_reduced15)
venues_reduced_10 = elimina_venue(venues,word_list_reduced10)

10145
11015


In [13]:
print(len(venues_reduced_15))
print(len(venues_reduced_10))

3810
3810


In [18]:
venues_final_15 = eliminare_duplicate_si_vide(venues_reduced_15)
venues_final_10 = eliminare_duplicate_si_vide(venues_reduced_10)
print(len(venues_final_15))
print(len(venues_final_10))

1626
1385


In [19]:
print_ll(venues_final_15, "venues_reduced_15")
print_ll(venues_final_10, "venues_reduced_10")